<a href="https://colab.research.google.com/github/takedatmh/toyama/blob/main/Adapter_merge_quick_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
LoRAマージ - 今すぐ実行できる最小構成コード
CPUでも動作し、実際のHuggingFaceアダプターを使用可能
"""

# 必要なパッケージのインストール（初回のみ）
# pip install transformers peft torch --upgrade

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, LoraConfig, get_peft_model, TaskType
import warnings
warnings.filterwarnings('ignore')

print("LoRAマージ - 即座に実行可能なコード")
print("=" * 50)

# ===========================
# 実行例1: 最軽量モデルで基本的なマージ
# ===========================

print("\n### 例1: OPT-125Mでの基本マージ（CPU可、メモリ500MB程度）")

# モデルのロード
print("モデルをロード中...")
base_model = AutoModelForCausalLM.from_pretrained("facebook/opt-125m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125m")

# LoRAアダプター設定
peft_config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

# PEFTモデル作成
model = get_peft_model(base_model, peft_config)
print(f"LoRAパラメータ数: {model.get_nb_trainable_parameters()[0]:,}")

# 2つ目のアダプターを追加
model.add_adapter("adapter2", peft_config)

# アダプターのマージ実行
print("\nアダプターをマージ中...")
model.add_weighted_adapter(
    adapters=["default", "adapter2"],
    weights=[0.6, 0.4],
    adapter_name="merged",
    combination_type="linear"
)

# マージしたアダプターで生成
model.set_adapter("merged")
prompt = "AI technology is"
inputs = tokenizer(prompt, return_tensors="pt")

print(f"\nプロンプト: '{prompt}'")
print("生成中...")

with torch.no_grad():
    outputs = model.generate(**inputs, max_length=30, temperature=0.8, do_sample=True)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"結果: {result}")

# ===========================
# 実行例2: GPT-2での実用的な例
# ===========================

print("\n\n### 例2: GPT-2でのマージ例（CPU可、メモリ1.5GB程度）")

# GPT-2モデル（日本語でも英語でも動作）
print("GPT-2をロード中...")
gpt2_model = AutoModelForCausalLM.from_pretrained("gpt2")
gpt2_tokenizer = AutoTokenizer.from_pretrained("gpt2")
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

# GPT-2用のLoRA設定
gpt2_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["c_attn", "c_proj"],  # GPT-2のアーキテクチャに対応
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

# PEFT model creation with the first adapter ("default")
gpt2_peft = get_peft_model(gpt2_model, gpt2_config)

# Add additional adapters
adapters = {
    "technical": {"r": 8, "alpha": 32},
    "creative": {"r": 8, "alpha": 16},
    "balanced": {"r": 8, "alpha": 24}
}

for name, params in adapters.items():
    config = LoraConfig(
        r=params["r"],
        lora_alpha=params["alpha"],
        target_modules=["c_attn", "c_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type=TaskType.CAUSAL_LM,
    )
    # Add adapter with the specified name
    gpt2_peft.add_adapter(name, config)


# 様々なマージ戦略
merge_strategies = [
    {
        "name": "tech_creative_mix",
        "adapters": ["default", "technical", "creative"],
        "weights": [0.2, 0.5, 0.3],
        "method": "linear"
    },
    {
        "name": "all_balanced",
        "adapters": ["default", "technical", "creative", "balanced"],
        "weights": [0.25, 0.25, 0.25, 0.25],
        "method": "linear"
    }
]

# マージ実行とテスト
test_prompts = [
    "The future of artificial intelligence",
    "How to write better code",
    "Once upon a time in a digital world"
]

for strategy in merge_strategies:
    print(f"\n戦略: {strategy['name']}")

    # マージ実行
    gpt2_peft.add_weighted_adapter(
        adapters=strategy["adapters"],
        weights=strategy["weights"],
        adapter_name=strategy["name"],
        combination_type=strategy["method"]
    )

    gpt2_peft.set_adapter(strategy["name"])

    # 最初のプロンプトでテスト
    prompt = test_prompts[0]
    inputs = gpt2_tokenizer(prompt, return_tensors="pt")

    with torch.no_grad():
        outputs = gpt2_peft.generate(
            **inputs,
            max_length=40,
            temperature=0.7,
            do_sample=True,
            pad_token_id=gpt2_tokenizer.eos_token_id
        )

    result = gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"  入力: {prompt}")
    print(f"  出力: {result}")

# ===========================
# 実行例3: 異なるマージ手法の比較
# ===========================

print("\n\n### 例3: マージ手法の比較")

# 新しいベースモデル
comparison_model = AutoModelForCausalLM.from_pretrained("facebook/opt-125m")
comparison_peft = get_peft_model(comparison_model, peft_config)

# アダプターを追加
for i in range(2):
    comparison_peft.add_adapter(f"adapter_{i+1}", peft_config)

# 異なるマージ手法を試す
methods_to_test = {
    "linear": {},
    "cat": {},  # concatenation
    "ties": {"density": 0.5, "majority_sign_method": "frequency"},
    "dare_linear": {"density": 0.8}
}

prompt = "Machine learning is"
print(f"\nテストプロンプト: '{prompt}'")
print("\n各マージ手法の結果:")

for method_name, extra_params in methods_to_test.items():
    try:
        # マージ実行
        merge_name = f"merged_{method_name}"
        comparison_peft.add_weighted_adapter(
            adapters=["default", "adapter_1"],
            weights=[0.6, 0.4],
            adapter_name=merge_name,
            combination_type=method_name,
            **extra_params
        )

        comparison_peft.set_adapter(merge_name)

        # 生成
        inputs = tokenizer(prompt, return_tensors="pt")
        with torch.no_grad():
            outputs = comparison_peft.generate(
                **inputs,
                max_length=25,
                temperature=0.7,
                do_sample=True
            )

        result = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"\n[{method_name}]: {result}")

    except Exception as e:
        print(f"\n[{method_name}]: エラー - {str(e)[:50]}...")

# ===========================
# 実行例4: 実用的なヘルパー関数
# ===========================

def quick_lora_merge(
    model_name="facebook/opt-125m",
    adapter_configs=None,
    merge_weights=None,
    merge_method="linear",
    test_prompt="Hello, AI!"
):
    """
    LoRAマージを簡単に実行するヘルパー関数

    Args:
        model_name: ベースモデル名
        adapter_configs: アダプター設定のリスト
        merge_weights: マージ時の重み
        merge_method: マージ手法
        test_prompt: テスト用プロンプト

    Returns:
        生成されたテキスト
    """
    # デフォルト設定
    if adapter_configs is None:
        adapter_configs = [
            {"r": 4, "alpha": 16},
            {"r": 4, "alpha": 32}
        ]

    if merge_weights is None:
        merge_weights = [0.5] * len(adapter_configs)

    # モデルロード
    base = AutoModelForCausalLM.from_pretrained(model_name)
    tok = AutoTokenizer.from_pretrained(model_name)

    # 最初のアダプター
    config = LoraConfig(
        r=adapter_configs[0]["r"],
        lora_alpha=adapter_configs[0]["alpha"],
        target_modules=["q_proj", "v_proj"],
        lora_dropout=0.1,
        bias="none",
        task_type=TaskType.CAUSAL_LM,
    )

    peft_model = get_peft_model(base, config)

    # 追加アダプター
    adapter_names = ["default"]
    for i, cfg in enumerate(adapter_configs[1:], 1):
        name = f"adapter_{i}"
        adapter_names.append(name)
        config = LoraConfig(
            r=cfg["r"],
            lora_alpha=cfg["alpha"],
            target_modules=["q_proj", "v_proj"],
            lora_dropout=0.1,
            bias="none",
            task_type=TaskType.CAUSAL_LM,
        )
        peft_model.add_adapter(name, config)

    # マージ
    peft_model.add_weighted_adapter(
        adapters=adapter_names,
        weights=merge_weights,
        adapter_name="final_merge",
        combination_type=merge_method
    )

    peft_model.set_adapter("final_merge")

    # 生成
    inputs = tok(test_prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = peft_model.generate(**inputs, max_length=50, temperature=0.8)

    return tok.decode(outputs[0], skip_special_tokens=True)

# ヘルパー関数の使用例
print("\n\n### 例4: 実用的なヘルパー関数の使用")

result = quick_lora_merge(
    adapter_configs=[
        {"r": 8, "alpha": 16},
        {"r": 8, "alpha": 32},
        {"r": 8, "alpha": 24}
    ],
    merge_weights=[0.5, 0.3, 0.2],
    merge_method="linear",
    test_prompt="The secret to happiness is"
)

print(f"ヘルパー関数の結果:\n{result}")

# ===========================
# 実行完了
# ===========================

print("\n" + "="*50)
print("すべての例が完了しました！")
print("\n実行環境:")
print(f"- PyTorch バージョン: {torch.__version__}")
print(f"- デバイス: {'CUDA' if torch.cuda.is_available() else 'CPU'}")
print(f"- メモリ効率: すべての例がCPUで実行可能")

print("\n💡 ヒント:")
print("1. より大きなモデルを試す場合は、load_in_8bit=True を使用")
print("2. 実際の訓練済みアダプターはHuggingFace Hubで検索可能")
print("3. マージ手法を変えることで異なる特性が得られます")
print("   - linear: バランスの取れた結合")
print("   - cat: 完全な機能保持（メモリ使用増）")
print("   - ties: スパース性を考慮した結合")
print("   - dare_linear: ランダム性を加えた結合")

LoRAマージ - 即座に実行可能なコード

### 例1: OPT-125Mでの基本マージ（CPU可、メモリ500MB程度）
モデルをロード中...
LoRAパラメータ数: 147,456

アダプターをマージ中...

プロンプト: 'AI technology is'
生成中...
結果: AI technology is a "thought-driven" process. We live with artificial intelligence, and we live with it. Today, the tech giant IBM


### 例2: GPT-2でのマージ例（CPU可、メモリ1.5GB程度）
GPT-2をロード中...

戦略: tech_creative_mix
  入力: The future of artificial intelligence
  出力: The future of artificial intelligence is unclear, but the company is set to unveil a new platform to take advantage of the high-tech capabilities of the next generation of AI.

The new platform,

戦略: all_balanced
  入力: The future of artificial intelligence
  出力: The future of artificial intelligence and the future of the world is in flux.

This post originally appeared on Digital Trends.

We are excited to announce that Samsung has finally created a new smart


### 例3: マージ手法の比較

テストプロンプト: 'Machine learning is'

各マージ手法の結果:

[linear]: Machine learning is the most efficient way to learn, and

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


ヘルパー関数の結果:
The secret to happiness is to be happy.
I'm not sure if you're being sarcastic or not, but I think you're right.

✅ すべての例が完了しました！

実行環境:
- PyTorch バージョン: 2.6.0+cu124
- デバイス: CUDA
- メモリ効率: すべての例がCPUで実行可能

💡 ヒント:
1. より大きなモデルを試す場合は、load_in_8bit=True を使用
2. 実際の訓練済みアダプターはHuggingFace Hubで検索可能
3. マージ手法を変えることで異なる特性が得られます
   - linear: バランスの取れた結合
   - cat: 完全な機能保持（メモリ使用増）
   - ties: スパース性を考慮した結合
   - dare_linear: ランダム性を加えた結合


このコードは、**LoRA（Low-Rank Adaptation）アダプターのマージ技術**を実演するもので、複数のLoRAアダプターを組み合わせて新しいモデルの振る舞いを作り出す方法を示しています。

## コードの主要な構成

### 1. **LoRAアダプターの基本設定パラメータ**

```python
peft_config = LoraConfig(
    r=4,                              # ランク（低ランク分解の次元数）
    lora_alpha=16,                    # スケーリングファクター
    target_modules=["q_proj", "v_proj"], # 適用対象のモジュール
    lora_dropout=0.1,                 # ドロップアウト率
    bias="none",                      # バイアスの扱い
    task_type=TaskType.CAUSAL_LM,    # タスクタイプ
)
```

### 2. **各パラメータの詳細解説**

#### **`r`（ランク）**
- 低ランク行列分解の次元数を指定
- 小さい値（4-8）：メモリ効率的だが表現力は限定的
- 大きい値（16-32）：より豊かな表現が可能だがメモリ使用量増加
- 例：`r=4`は元の重み行列を4次元の低ランク行列で近似

#### **`lora_alpha`（アルファ）**
- LoRAの更新をスケールする係数
- 実質的な学習率のような役割：`lora_alpha / r`
- 例：`r=4, lora_alpha=16`の場合、実効的なスケーリングは4倍

#### **`target_modules`（対象モジュール）**
- LoRAを適用する具体的なレイヤー/モジュール
- モデルアーキテクチャごとに異なる：
  - OPT/LLaMA: `["q_proj", "v_proj"]`（Query/Value投影）
  - GPT-2: `["c_attn", "c_proj"]`（Attention/Projection）
- これらは自己注意機構の重要な部分

### 3. **マージ戦略の実装例**

```python
# 複数アダプターの重み付きマージ
model.add_weighted_adapter(
    adapters=["default", "adapter2"],  # マージするアダプター名
    weights=[0.6, 0.4],               # 各アダプターの重み
    adapter_name="merged",            # 新しいアダプター名
    combination_type="linear"         # マージ手法
)
```

### 4. **異なるマージ手法**

コードでは4つのマージ手法を比較：

- **`linear`**: 単純な線形結合（重み付き平均）
- **`cat`**: 連結（全機能保持、メモリ増）
- **`ties`**: スパース性考慮（密度パラメータ付き）
- **`dare_linear`**: ランダム性を加えた線形結合

### 5. **実用的な使用例**

```python
# GPT-2での複数アダプター設定
adapters = {
    "technical": {"r": 8, "alpha": 32},  # 技術的な文章用
    "creative": {"r": 8, "alpha": 16},   # 創造的な文章用
    "balanced": {"r": 8, "alpha": 24}    # バランス型
}
```

各アダプターは異なる`alpha`値を持ち、異なる「性格」を表現：
- **technical**: 高いalpha（32）で強い影響
- **creative**: 低いalpha（16）で柔軟な表現
- **balanced**: 中間的な値（24）

### 6. **マージの実行フロー**

1. ベースモデルをロード
2. 初期LoRAアダプターを作成
3. 追加のアダプターを作成（異なる設定で）
4. 重み付きマージを実行
5. マージしたアダプターでテキスト生成

このコードの優れた点は、**CPU環境でも動作**し、メモリ効率的に複数のLoRAアダプターを組み合わせられることです。実際の用途では、異なるタスクで訓練されたアダプターをマージして、多機能なモデルを作成できます。